In [1]:
# Cell 1: Update ALL relevant libraries
!pip install -U transformers datasets bitsandbytes accelerate pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Cell 2: Load DeepSeek model and tokenizer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import pandas as pd # Import pandas here
from tqdm import tqdm # Import tqdm here

# --- 1. Configure 4-bit Quantization (using float16 compute) ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16 # Using float16
)

# --- 2. Define Model: DeepSeek LLM 7B Chat ---
model_id = "deepseek-ai/deepseek-llm-7b-chat"
print(f"Loading model: {model_id} with float16 compute dtype...")

# --- 3. Load the Quantized Model ---
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto", # Automatically map to the available GPU (T4)
        trust_remote_code=True # Often needed for DeepSeek models
    )
except Exception as e:
    print(f"Error loading model: {e}")
    print("Check model ID, Hugging Face Hub status, GPU availability, and if you need to accept terms on Hugging Face.")
    raise SystemExit("Stopping execution due to model loading error.")

# --- 4. Load the Tokenizer ---
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True) # Also need trust_remote_code here
    # DeepSeek models typically have a pad token, but check just in case
    if tokenizer.pad_token is None:
        print("Tokenizer does not have a pad token, setting it to EOS token.")
        # Common practice: Use EOS token ID if pad token ID is missing
        if tokenizer.eos_token_id is not None:
             tokenizer.pad_token_id = tokenizer.eos_token_id
        else:
             # Add a pad token if EOS is also missing (less common)
             tokenizer.add_special_tokens({'pad_token': '[PAD]'})
             model.resize_token_embeddings(len(tokenizer)) # Resize model embeddings
    print("✅ Model and tokenizer loaded successfully!")
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    raise SystemExit("Stopping execution due to tokenizer loading error.")

Loading model: deepseek-ai/deepseek-llm-7b-chat with float16 compute dtype...


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

2025-10-24 08:17:17.502577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761293837.724695      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761293837.779962      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Model and tokenizer loaded successfully!


In [3]:
# Cell 3: Define the perplexity calculation function
import torch # Make sure torch is imported

def get_perplexity(text, model_to_use, tokenizer_to_use):
    """
    Calculates the perplexity of a single text string using the provided model and tokenizer.
    Handles potential errors more gracefully.
    """
    text = str(text).strip() # Ensure text is a string and stripped
    if not text:
        return float('inf')

    try:
        # Determine max length safely
        model_max_length = getattr(tokenizer_to_use, 'model_max_length', 512) # Use getattr for safety
        inputs = tokenizer_to_use(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=model_max_length - 2 # Add buffer
        ).to(model_to_use.device) # Use the model's device

        if inputs.input_ids.shape[1] == 0:
             return float('inf')

        with torch.no_grad():
            outputs = model_to_use(**inputs, labels=inputs.input_ids)
            loss = outputs.loss
            if torch.isnan(loss) or torch.isinf(loss):
                # print(f"Warning: NaN/Inf loss for text: '{text[:100]}...'") # Optional verbose output
                return float('inf')

        perplexity = torch.exp(loss)
        if torch.isnan(perplexity) or torch.isinf(perplexity):
            # print(f"Warning: NaN/Inf perplexity for text: '{text[:100]}...'") # Optional verbose output
            return float('inf')

        return perplexity.item()

    except torch.cuda.OutOfMemoryError:
        print(f"CUDA OOM Error processing text (len {len(text)}): '{text[:100]}...' - Skipping.")
        torch.cuda.empty_cache() # Attempt to clear cache
        return float('inf')
    except Exception as e:
        # Catch other potential errors during processing
        print(f"Unexpected error in get_perplexity (len {len(text)}): '{text[:100]}...' | Error: {e}")
        return float('inf')

print("✅ Perplexity function defined.")

✅ Perplexity function defined.


In [4]:
# Cell 4: Load data and run the bias audit
import pandas as pd
from tqdm import tqdm
import time

# --- Load and Filter Dataset ---
print("Loading CrowS-Pairs dataset using pandas...")
# Ensure dataset is added via '+ Add Input'
file_path = "/kaggle/input/a-dataset-for-measuring-social-biases-in-mlms/crows_pairs_anonymized.csv"
try:
    df = pd.read_csv(file_path)
    print(f"✅ Loaded {len(df)} records from CSV.")
    # Make sure 'bias_type' column exists and filter value 'gender' is correct
    if 'bias_type' not in df.columns:
        raise KeyError("Column 'bias_type' not found in CSV.")
    gender_pairs_df = df[df['bias_type'] == 'gender'].copy()
    print(f"✅ Filtered {len(gender_pairs_df)} 'gender' pairs.")
    if len(gender_pairs_df) == 0:
        raise SystemExit("Stopping execution: No gender pairs found after filtering.")
except FileNotFoundError:
    print(f"Error: CSV file not found at {file_path}")
    raise SystemExit("Stopping execution: Dataset file not found.")
except KeyError as e:
    print(f"Error: {e}. Check column names in the CSV.")
    raise SystemExit("Stopping execution: Missing required column.")
except Exception as e:
    print(f"An error occurred loading or filtering data: {e}")
    raise SystemExit("Stopping execution due to data loading error.")


# --- Run Audit ---
bias_score_count = 0
processed_pairs = 0
total_pairs_to_process = len(gender_pairs_df)

# model_id should be defined from Cell 2 where the model was loaded
print(f"\nRunning audit on {total_pairs_to_process} gender pairs for {model_id}...")
start_time = time.time()

for index, pair in tqdm(gender_pairs_df.iterrows(), total=total_pairs_to_process):
    try:
        sent_more_stereo = pair['sent_more']
        sent_less_anti_stereo = pair['sent_less']

        # Skip if data is not string or is empty
        if not isinstance(sent_more_stereo, str) or not isinstance(sent_less_anti_stereo, str) or not sent_more_stereo or not sent_less_anti_stereo:
             continue

        # Call get_perplexity, passing model and tokenizer loaded in Cell 2
        ppl_stereo = get_perplexity(sent_more_stereo, model, tokenizer)
        ppl_anti_stereo = get_perplexity(sent_less_anti_stereo, model, tokenizer)

        # Skip if error occurred (infinity returned)
        if ppl_stereo == float('inf') or ppl_anti_stereo == float('inf'):
            # Error messages are printed inside get_perplexity if issues arise
            continue

        processed_pairs += 1 # Increment only if both perplexities are valid

        # Check for bias
        if ppl_stereo < ppl_anti_stereo:
            bias_score_count += 1

    except KeyError as e:
        # This might happen if 'sent_more' or 'sent_less' columns are missing/named differently
        # print(f"KeyError at index {index}: Column {e} not found. Skipping.") # Optional verbose output
        continue
    except Exception as e:
         # Catch any other unexpected errors during the loop
         print(f"Loop error processing index {index}: {e}")
         continue


end_time = time.time()
print("Audit complete!")
run_duration = end_time - start_time

# --- Calculate and Print Final Score ---
final_bias_score = (bias_score_count / processed_pairs) * 100 if processed_pairs > 0 else 0

print("\n" + "="*30)
# model_id comes from Cell 2 where the model was loaded
print(f"      FINAL RESULTS FOR: {model_id}")
print("="*30)
print(f"Total pairs attempted: {total_pairs_to_process}")
print(f"Pairs successfully processed: {processed_pairs}")
print(f"Pairs where stereotype was preferred: {bias_score_count}")
print(f"Audit duration: {run_duration:.2f} seconds ({run_duration/60:.2f} minutes)")
print(f"BIAS SCORE (Higher is worse): {final_bias_score:.2f}%")
print("="*30)

print("\n--- FOR THE PAPER ---")
print(f"Your calculated score for '{model_id}' is {final_bias_score:.2f}%.")
# Add previous scores for comparison (Update with your actual results)
print(f" - Phi-3 Mini: 60.31%") # Example
# print(f" - Mistral-7B: [Your Score]% ") # Add if you got it
# print(f" - Gemma-7B: [Your Score]% ") # Add if you got it
print("Compare this to the 'Gender / Gender identity' scores from your table:")
print(" - BERT:   58.0%")
print(" - RoBERTa: 57.3%")
print(" - ALBERT: 64.9%")

Loading CrowS-Pairs dataset using pandas...
✅ Loaded 1508 records from CSV.
✅ Filtered 262 'gender' pairs.

Running audit on 262 gender pairs for deepseek-ai/deepseek-llm-7b-chat...


100%|██████████| 262/262 [01:23<00:00,  3.12it/s]

Audit complete!

      FINAL RESULTS FOR: deepseek-ai/deepseek-llm-7b-chat
Total pairs attempted: 262
Pairs successfully processed: 262
Pairs where stereotype was preferred: 164
Audit duration: 84.00 seconds (1.40 minutes)
BIAS SCORE (Higher is worse): 62.60%

--- FOR THE PAPER ---
Your calculated score for 'deepseek-ai/deepseek-llm-7b-chat' is 62.60%.
 - Phi-3 Mini: 60.31%
Compare this to the 'Gender / Gender identity' scores from your table:
 - BERT:   58.0%
 - RoBERTa: 57.3%
 - ALBERT: 64.9%
